In [137]:

import mne 
import glob

#matplotlib.use('Qt5Agg')

In [138]:
files = glob.glob("eeg_data_task_rest/*.edf")
files

['eeg_data_task_rest\\Subject00_1.edf',
 'eeg_data_task_rest\\Subject00_2.edf',
 'eeg_data_task_rest\\Subject01_1.edf',
 'eeg_data_task_rest\\Subject01_2.edf',
 'eeg_data_task_rest\\Subject02_1.edf',
 'eeg_data_task_rest\\Subject02_2.edf',
 'eeg_data_task_rest\\Subject03_1.edf',
 'eeg_data_task_rest\\Subject03_2.edf',
 'eeg_data_task_rest\\Subject04_1.edf',
 'eeg_data_task_rest\\Subject04_2.edf',
 'eeg_data_task_rest\\Subject05_1.edf',
 'eeg_data_task_rest\\Subject05_2.edf',
 'eeg_data_task_rest\\Subject06_1.edf',
 'eeg_data_task_rest\\Subject06_2.edf',
 'eeg_data_task_rest\\Subject07_1.edf',
 'eeg_data_task_rest\\Subject07_2.edf',
 'eeg_data_task_rest\\Subject08_1.edf',
 'eeg_data_task_rest\\Subject08_2.edf',
 'eeg_data_task_rest\\Subject09_1.edf',
 'eeg_data_task_rest\\Subject09_2.edf',
 'eeg_data_task_rest\\Subject10_1.edf',
 'eeg_data_task_rest\\Subject10_2.edf',
 'eeg_data_task_rest\\Subject11_1.edf',
 'eeg_data_task_rest\\Subject11_2.edf',
 'eeg_data_task_rest\\Subject12_1.edf',


In [139]:
rest_files = [i for i in files if '1' in i[-5]]
task_files = [i for i in files if '2' in i[-5]]
print(len(rest_files), len(task_files))

36 36


In [140]:
def my_read_raw(path):
    raw = mne.io.read_raw(path, preload=True)
    raw_filtered = raw.filter(l_freq = 0.5, h_freq = 60)
    return raw_filtered

In [141]:
import numpy as np
import matplotlib.pyplot as plt

# Define the frequency bands
freq_bands = {
    'Delta': (1, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 30),
    'Gamma': (30, 60)
}

In [144]:
import matplotlib_inline
import matplotlib.pyplot as plt
def calculate_psd(raw_data, fmin, fmax):
    psd, freqs = mne.time_frequency.psd_array_welch(raw_data.get_data(), sfreq=raw_data.info['sfreq'], fmin=fmin, fmax=fmax, n_fft=2048)
    return psd, freqs

def plot_psd(psd, freqs, title):
    plt.figure(figsize=(10, 6))
    plt.semilogy(freqs, psd.T)
    plt.title(title)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power Spectral Density (dB)')
    plt.show()

def my_plot_psd(raw_rest, raw_task):
    # Calculate and plot PSD for each frequency band for both states
    for band, (fmin, fmax) in freq_bands.items():
        rest_psd, rest_freqs = calculate_psd(raw_rest, fmin, fmax)
        task_psd, task_freqs = calculate_psd(raw_task, fmin, fmax)
        
        plot_psd(rest_psd, rest_freqs, f'{band} Band - Rest State')
        plot_psd(task_psd, task_freqs, f'{band} Band - Task State')

my_plot_psd gives seperate plots for rest and task states while plotting each channel, but this isnt much comfortable to analyse visually by graphs so we need to take mean w.r.t channels to get a mean psd and plot it in same graph to analyse properly
this work will be done by compare_plot_psd function

In [145]:
def compare_plot_psd(rest_psd, task_psd, freqs, band):
    #print(rest_psd.shape)
    mean_rest_psd = np.mean(rest_psd, axis=0)
    mean_task_psd = np.mean(task_psd, axis=0)
    #print(mean_rest_psd)
    #print(mean_rest_psd.shape)
    plt.figure(figsize=(10, 6))
    plt.semilogy(freqs, mean_rest_psd, label='Rest State')
    plt.semilogy(freqs, mean_task_psd, label='Task State')
    plt.title(f'Comparison of PSD in {band} Band')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power Spectral Density (dB)')
    plt.legend()
    plt.show()

def compare_psd(rest_raw, task_raw):
    # Compare PSDs for each frequency band
    mean_psd_dict = {}
    for band, (fmin, fmax) in freq_bands.items():
        rest_psd, rest_freqs = calculate_psd(rest_raw, fmin, fmax)
        task_psd, task_freqs = calculate_psd(task_raw, fmin, fmax)
        #get mean for each channel and each frequency of psd
        mean_for_channel_rest = np.mean(rest_psd, axis=0)
        mean_for_channel_task = np.mean(task_psd, axis=0)
        final_mean_rest = np.mean(mean_for_channel_rest)
        final_mean_task = np.mean(mean_for_channel_task)
        mean_psd_dict[band+"_rest"] = final_mean_rest
        mean_psd_dict[band+"_task"] = final_mean_task
        compare_plot_psd(rest_psd, task_psd, rest_freqs, band)
    return mean_psd_dict

we will also be needing to save psd avg for each band so we will be considering mean only

In [146]:
%%capture
raw_rest_list = [my_read_raw(i) for i in rest_files]
raw_task_list = [my_read_raw(i) for i in task_files]

In [147]:
a1 = compare_psd(raw_rest_list[30], raw_task_list[30])

Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)


In [148]:
my_plot_psd(raw_rest_list[0], raw_task_list[0])

Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)


In [149]:
a1

{'Delta_rest': 2.6447589302678295e-11,
 'Delta_task': 2.7620993558175563e-11,
 'Theta_rest': 3.046598712034397e-11,
 'Theta_task': 3.442945575988138e-11,
 'Alpha_rest': 2.1839873234322315e-11,
 'Alpha_task': 2.0007902423862475e-11,
 'Beta_rest': 9.559482940626906e-12,
 'Beta_task': 9.987148222231333e-12,
 'Gamma_rest': 3.9807252078783045e-13,
 'Gamma_task': 4.3129987477712614e-13}

Now I will be storing it in a dataframe for each subject

In [150]:
files[1].split("\\")[1].split("_")[0]

'Subject00'

get list [Subject00, Subject01, ....]

In [151]:
subjects = [file.split("\\")[1].split("_")[0] for file in rest_files]
subjects

['Subject00',
 'Subject01',
 'Subject02',
 'Subject03',
 'Subject04',
 'Subject05',
 'Subject06',
 'Subject07',
 'Subject08',
 'Subject09',
 'Subject10',
 'Subject11',
 'Subject12',
 'Subject13',
 'Subject14',
 'Subject15',
 'Subject16',
 'Subject17',
 'Subject18',
 'Subject19',
 'Subject20',
 'Subject21',
 'Subject22',
 'Subject23',
 'Subject24',
 'Subject25',
 'Subject26',
 'Subject27',
 'Subject28',
 'Subject29',
 'Subject30',
 'Subject31',
 'Subject32',
 'Subject33',
 'Subject34',
 'Subject35']

This is just the above function but it doesnt draw comparison graphs but returns psd data

In [152]:
def compare_psd_dict(rest_raw, task_raw):
    # Compare PSDs for each frequency band
    mean_psd_dict = {}
    for band, (fmin, fmax) in freq_bands.items():
        rest_psd, rest_freqs = calculate_psd(rest_raw, fmin, fmax)
        task_psd, task_freqs = calculate_psd(task_raw, fmin, fmax)
        #get mean for each channel and each frequency of psd
        mean_for_channel_rest = np.mean(rest_psd, axis=0)
        mean_for_channel_task = np.mean(task_psd, axis=0)
        final_mean_rest = np.mean(mean_for_channel_rest)
        final_mean_task = np.mean(mean_for_channel_task)
        mean_psd_dict[band+"_rest"] = final_mean_rest
        mean_psd_dict[band+"_task"] = final_mean_task
    return mean_psd_dict

In [153]:
def fetch_psd_band(rest_list, task_list):
    final_dict = {}
    temp_dict = compare_psd_dict(rest_list[0], task_list[0])
    print(temp_dict)
    for index, (key, value) in enumerate(temp_dict.items()):
        final_dict[key] = [value]
    for i in range(1, len(rest_list)):
        temp_dict = compare_psd_dict(rest_list[i], task_list[i])
        print(temp_dict)
        for index, (key, value) in enumerate(temp_dict.items()):
            final_dict[key].append(value)
    return final_dict

In [154]:
df_dict = {}
df_dict['subjects'] = subjects
df_dict

{'subjects': ['Subject00',
  'Subject01',
  'Subject02',
  'Subject03',
  'Subject04',
  'Subject05',
  'Subject06',
  'Subject07',
  'Subject08',
  'Subject09',
  'Subject10',
  'Subject11',
  'Subject12',
  'Subject13',
  'Subject14',
  'Subject15',
  'Subject16',
  'Subject17',
  'Subject18',
  'Subject19',
  'Subject20',
  'Subject21',
  'Subject22',
  'Subject23',
  'Subject24',
  'Subject25',
  'Subject26',
  'Subject27',
  'Subject28',
  'Subject29',
  'Subject30',
  'Subject31',
  'Subject32',
  'Subject33',
  'Subject34',
  'Subject35']}

In [155]:
df_dict = fetch_psd_band(raw_rest_list, raw_task_list)

Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
{'Delta_rest': 1.1076912536304064e-10, 'Delta_task': 5.892148983376907e-11, 'Theta_rest': 4.231894479779173e-11, 'Theta_task': 3.6419706745714825e-11, 'Alpha_rest': 4.616463989081962e-11, 'Alpha_task': 5.360880819652999e-11, 'Beta_rest': 8.801239214079302e-12, 'Beta_task': 9.876687997633415e-12, 'Gamma_rest': 1.3030379180511252e-13, 'Gamma_task': 1.722552405779942e-13}
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window size : 4.096 (s)
Effective window

In [156]:
df_dict

{'Delta_rest': [1.1076912536304064e-10,
  5.082642142436406e-11,
  1.6716980025126606e-11,
  2.5458555953163145e-11,
  6.862068264498865e-11,
  1.1001163564817474e-10,
  6.289079449340027e-11,
  2.7965536191820925e-11,
  1.508130240358583e-10,
  1.1050182507781185e-10,
  6.739149178248161e-11,
  6.228412106282988e-10,
  1.2775975773350964e-10,
  7.384985850681531e-12,
  8.964154694195484e-11,
  9.906833770010654e-11,
  2.7451783701674238e-11,
  4.495152406352684e-11,
  8.923714019264094e-11,
  5.779005385302433e-11,
  5.976295340360395e-11,
  1.0158997399517387e-10,
  6.672610877280313e-11,
  2.096477091854093e-11,
  8.420783224748913e-11,
  7.328421265735628e-11,
  6.697905983161712e-11,
  6.829170312036116e-11,
  3.9889215286184475e-11,
  1.0377906979227119e-10,
  2.6447589302678295e-11,
  7.923129955741524e-11,
  6.803636585806454e-11,
  2.3202258259867187e-11,
  6.687527898063467e-11,
  8.564127872304422e-11],
 'Delta_task': [5.892148983376907e-11,
  6.070404913574334e-11,
  6.7048

In [157]:
pos = list(df_dict.keys()).index('Delta_rest')
print(pos)
items = list(df_dict.items())
items.insert(pos, 'subjects')
df_dict['subjects'] = subjects
df_dict

0


{'Delta_rest': [1.1076912536304064e-10,
  5.082642142436406e-11,
  1.6716980025126606e-11,
  2.5458555953163145e-11,
  6.862068264498865e-11,
  1.1001163564817474e-10,
  6.289079449340027e-11,
  2.7965536191820925e-11,
  1.508130240358583e-10,
  1.1050182507781185e-10,
  6.739149178248161e-11,
  6.228412106282988e-10,
  1.2775975773350964e-10,
  7.384985850681531e-12,
  8.964154694195484e-11,
  9.906833770010654e-11,
  2.7451783701674238e-11,
  4.495152406352684e-11,
  8.923714019264094e-11,
  5.779005385302433e-11,
  5.976295340360395e-11,
  1.0158997399517387e-10,
  6.672610877280313e-11,
  2.096477091854093e-11,
  8.420783224748913e-11,
  7.328421265735628e-11,
  6.697905983161712e-11,
  6.829170312036116e-11,
  3.9889215286184475e-11,
  1.0377906979227119e-10,
  2.6447589302678295e-11,
  7.923129955741524e-11,
  6.803636585806454e-11,
  2.3202258259867187e-11,
  6.687527898063467e-11,
  8.564127872304422e-11],
 'Delta_task': [5.892148983376907e-11,
  6.070404913574334e-11,
  6.7048

In [158]:
import pandas as pd 

df = pd.DataFrame(df_dict)
df.head()

,Delta_rest,Delta_task,Theta_rest,Theta_task,Alpha_rest,Alpha_task,Beta_rest,Beta_task,Gamma_rest,Gamma_task,subjects
0,1.107691e-10,5.892149e-11,4.231894e-11,3.641971e-11,4.616464e-11,5.360881e-11,8.801239e-12,9.876688e-12,1.303038e-13,1.722552e-13,Subject00
1,5.082642e-11,6.070405e-11,2.099879e-11,1.513141e-11,1.478213e-11,1.092669e-11,3.413762e-12,3.026062e-12,3.210538e-13,4.526791e-13,Subject01
2,1.671698e-11,6.704840e-11,3.387951e-12,1.858657e-11,7.418611e-12,1.450427e-11,1.485142e-12,8.592887e-12,1.932228e-13,7.568279e-13,Subject02
3,2.545856e-11,2.522412e-11,1.650008e-11,1.423979e-11,2.307572e-11,1.440940e-11,3.434753e-12,3.428508e-12,1.850666e-13,2.184036e-13,Subject03
4,6.862068e-11,5.553461e-11,1.722039e-11,1.778828e-11,1.986344e-11,1.618999e-11,5.478877e-12,5.412901e-12,4.448268e-13,1.554313e-13,Subject04


Plot each subject psd mean in a comparitive plot for each band

In [159]:

# Selecting only the relevant columns for plotting
df_plot = df[['subjects', 'Delta_rest', 'Delta_task']]
# Plotting the comparative bar graph
ax = df_plot.plot(kind='bar', figsize=(10, 6))
plt.show()

In [160]:
# Selecting only the relevant columns for plotting
df_plot = df[['subjects', 'Theta_rest', 'Theta_task']]
# Plotting the comparative bar graph
ax = df_plot.plot(kind='bar', figsize=(10, 6))
plt.show()

In [161]:
# Selecting only the relevant columns for plotting
df_plot = df[['subjects', 'Alpha_rest', 'Alpha_task']]
# Plotting the comparative bar graph
ax = df_plot.plot(kind='bar', figsize=(10, 6))
plt.show()

In [162]:
# Selecting only the relevant columns for plotting
df_plot = df[['subjects', 'Beta_rest', 'Beta_task']]
# Plotting the comparative bar graph
ax = df_plot.plot(kind='bar', figsize=(10, 6))
plt.show()

In [163]:
# Selecting only the relevant columns for plotting
df_plot = df[['subjects', 'Gamma_rest', 'Gamma_task']]
# Plotting the comparative bar graph
ax = df_plot.plot(kind='bar', figsize=(10, 6))
plt.show()

The graphs are stored in graphs folder